In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from constants import *
import pyndri
import sqlite3
import numpy as np
import pickle
from retrieval import index as idx
from dataset.Dataset import Dataset

In [3]:
with sqlite3.connect(FILTERED_DB) as conn:
    cursor = conn.cursor()
    data = Dataset(TRAINING_SET)
    for question in data:
        cursor.execute(f"SELECT id from filtered WHERE id = ?", (question.id,))
        if cursor.fetchone() is None:
            print(question)

In [8]:
lengths = np.empty(len(data))
with sqlite3.connect(FILTERED_DB) as conn:
    cursor = conn.cursor()
    data = Dataset(TRAINING_SET)
    for i, question in enumerate(data):
        cursor.execute(f"SELECT result_int_ids from filtered WHERE id = ?", (question.id,))
        (result,) = cursor.fetchone()
        result = pickle.loads(result)
        lengths[i] = len(result)

In [9]:
lengths.mean()

2130.671431888288

In [ ]:
def accuracy(true_positives: int, all_observations: int) -> float:
    if all_observations == 0 and true_positives == 0:
        return 0

    return true_positives / all_observations

comparison = 0
bridge = 1
hard = 0
medium = 1
easy = 2

gold_article_ids = set()
[[gold_article_ids.add(index.external2internal(idx)) for idx in index.title2wid[title]] for title in
 question['gold']]

number_of_articles_found = len(gold_article_ids.intersection(filtered_articles))
question_type = comparison if question['type'] == 'comparison' else bridge
question_level = hard if question['level'] == 'hard' else medium if question['level'] == 'medium' else easy
found_articles[question_type][question_level][number_of_articles_found] += 1

# shape: (question type [comparison, bridge], level [hard, medium, easy], number of found gold articles [0, 1, 2])
found_articles = numpy.zeros((2, 3, 3))
fully_accurate = accuracy(int(numpy.sum(found_articles[:, :, 2])), int(numpy.sum(found_articles[:, :, :])))
somewhat_accurate = accuracy(int(numpy.sum(found_articles[:, :, 1:])), int(numpy.sum(found_articles[:, :, :])))
accurate_hard_questions = accuracy(int(numpy.sum(found_articles[:, hard, 2])),
                                   int(numpy.sum(found_articles[:, hard, :])))
accurate_medium_questions = accuracy(int(numpy.sum(found_articles[:, medium, 2])),
                                     int(numpy.sum(found_articles[:, medium, :])))
accurate_easy_questions = accuracy(int(numpy.sum(found_articles[:, easy, 2])),
                                   int(numpy.sum(found_articles[:, easy, :])))
accurate_comparison_questions = accuracy(int(numpy.sum(found_articles[comparison, :, 2])),
                                         int(numpy.sum(found_articles[comparison, :, :])))
accurate_bridge_questions = accuracy(int(numpy.sum(found_articles[bridge, :, 2])),
                                     int(numpy.sum(found_articles[bridge, :, :])))

print(f'Finished processing in {datetime.now() - start}')
print(f'Fully accurate: {round(fully_accurate, 4)}')
print(f'Somewhat accurate: {round(somewhat_accurate, 4)}')
print('-' * 10)
print(f'easy question accuracy: {round(accurate_easy_questions, 4)}')
print(f'medium question accuracy: {round(accurate_medium_questions, 4)}')
print(f'hard question accuracy: {round(accurate_hard_questions, 4)}')
print('-' * 10)
print(f'comparison question accuracy: {round(accurate_comparison_questions, 4)}')
print(f'bridge question accuracy: {round(accurate_bridge_questions, 4)}')

numpy.save(constants.FILTER_RESULTS, found_articles)